In [1]:
# Importing Pandas for data Wrangaing and numpy arthmetic operations.
import pandas as pd
import numpy as np

# Imporing NLTK for text preprocessing.
import nltk
nltk.download('stopwords')

import nltk
nltk.download('punkt')

from nltk.corpus import stopwords
import gensim  # Gensim for text vectorization.
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder

from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix


# To avoid warnings.
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
new_df = pd.read_csv('IMDB Dataset.csv')
new_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
new_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
df = new_df.iloc[ :20000] # takinhg only 20k reviews 

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     20000 non-null  object
 1   sentiment  20000 non-null  object
dtypes: object(2)
memory usage: 312.6+ KB


In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
# Seeing whether we have balanced or imbalanced data.
df['sentiment'].value_counts()

negative    10097
positive     9903
Name: sentiment, dtype: int64

In [8]:
df.duplicated().sum()

74

In [9]:

df[df.duplicated()]


,review,sentiment
3537,Quite what the producers of this appalling ada...,negative
3769,My favourite police series of all time turns t...,positive
4391,"Beautiful film, pure Cassavetes style. Gena Ro...",positive
6352,If you liked the Grinch movie... go watch that...,negative
6479,I want very much to believe that the above quo...,negative
...,...,...
19720,The problem with so many people watching this ...,positive
19827,"Goodnight, Mister Tom begins in an impossibly ...",negative
19861,This movie starts out as if it were a comedy. ...,negative
19890,"Humm, an Italian movie starred by David hassel...",negative


In [10]:
df.drop_duplicates(inplace = True) # droping the duplicate reviews

In [11]:
df['review'][3].lower()

"basically there's a family where a little boy (jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />this movie is slower than a soap opera... and suddenly, jake decides to become rambo and kill the zombie.<br /><br />ok, first of all when you're going to make a film you must decide if its a thriller or a drama! as a drama the movie is watchable. parents are divorcing & arguing like in real life. and then we have jake with his closet which totally ruins all the film! i expected to see a boogeyman similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. as for the shots with jake: just ignore them."

In [12]:
df['review']= df['review'].str.lower()
df

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
19995,"ok. for starters, taxi driver is amazing. this...",negative
19996,"it's sort of hard for me to say it, because i ...",negative
19997,i still liked it though. warren beatty is only...,positive
19998,we could still use black adder even today. ima...,positive


In [13]:
import re
def remove_html_tags(text):               # Removing the HTML tages
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [14]:
df['review']= df['review'].apply(remove_html_tags)

In [15]:
df['review'][1]

'a wonderful little production. the filming technique is very unassuming- very old-time-bbc fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. the actors are extremely well chosen- michael sheen not only "has got all the polari" but he has all the voices down pat too! you can truly see the seamless editing guided by the references to williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. a masterful production about one of the great master\'s of comedy and his life. the realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. it plays on our knowledge and our senses, particularly with the scenes concerning orton and halliwell and the sets (particularly of their flat with halliwell\'s murals decorating every surface) are terribly well done.'

In [16]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [17]:
exclude = string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [18]:
def remove_pun(text):              # Removing the Punctuations.
    return text.translate(str.maketrans('','',exclude))
# df['review'] = df['review'].apply(lambda x: ['' for item in x.split() if item in exclude]).apply(lambda x:" ".join(x))

In [19]:
df['review'].apply(remove_pun)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production the filming tech...
2        i thought this was a wonderful way to spend ti...
3        basically theres a family where a little boy j...
4        petter matteis love in the time of money is a ...
                               ...                        
19995    ok for starters taxi driver is amazing this th...
19996    its sort of hard for me to say it because i so...
19997    i still liked it though warren beatty is only ...
19998    we could still use black adder even today imag...
19999    this socalled documentary tries to tell that u...
Name: review, Length: 19926, dtype: object

In [20]:
df['review'][0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [21]:
chat_words = {'AFAIK':'As Far As I Know',
'AFK' :'Away From Keyboard',
'ASAP' :'As Soon As Possible',
'ATK': 'At The Keyboard',
'ATM':'At The Moment',
'A3': 'Anytime, Anywhere, Anyplace',
'BAK':'Back At Keyboard',
'BBL':'Be Back Later',
    
'BBS':'Be Back Soon',
'BFN':'Bye For Now',
'B4N':'Bye For Now',
'BRB':'Be Right Back',
'BRT':'Be Right There',
'BTW':'By The Way',
'B4':'Before',
'B4N':'Bye For Now',
'CU':'See You',
'CUL8R':'See You Later',
'CYA':'See You',
'FAQ':'Frequently Asked Questions',
'FC':'Fingers Crossed',
    
'FWIW':'For What It is Worth',
'FYI':'For Your Information',
'GAL':'Get A Life',
'GG':'Good Game',
              
              'GN':'Good Night',
'GMTA':'Great Minds Think Alike',
'GR8':'Great',
'G9':'Genius',
'IC':'I See',
'ICQ':'I Seek you (also a chat program)',
'ILU':'I Love You',
'IMHO':'In My Honest/Humble Opinion',
'IMO':'In My Opinion',
'IOW':'In Other Words',
'IRL':'In Real Life',
'KISS':'Keep It Simple, Stupid',
'LDR':'Long Distance Relationship',
'LMAO':'Laugh My A.. Off',
'LOL':'Laughing Out Loud',
'LTNS':'Long Time No See',
'L8R':'Later',
'MTE':'My Thoughts Exactly',
'M8':'Mate',
'NRN':'No Reply Necessary',
'OIC':'Oh I See',
'PITA':'Pain In The Ass',
'PRT':'Party',
'PRW':'Parents Are Watching',
'QPSA':'Que Pasa',
'ROFL':'Rolling On The Floor Laughing',
'ROFLOL':'Rolling On The Floor Laughing Out Loud',
'ROTFLMAO':'Rolling On The Floor Laughing My A.. Off',
    
'SK8':'Skate',
'STATS':'Your sex and age',
'ASL':'Age, Sex, Location',
'THX':'Thank You',
'TTFN':'Ta-Ta For Now',
'TTYL':'Talk To You Later',
'U':'You',
'U2':'You Too',
'U4E':'Yours For Ever',
'WB':'Welcome Back',
'TF':'What The F...',
'WTG':'Way To Go',
'WUF':'Where Are You From',
'W8':'Wait',
'7K':'Sick'}            
              

In [22]:
chat_words

{'AFAIK': 'As Far As I Know',
 'AFK': 'Away From Keyboard',
 'ASAP': 'As Soon As Possible',
 'ATK': 'At The Keyboard',
 'ATM': 'At The Moment',
 'A3': 'Anytime, Anywhere, Anyplace',
 'BAK': 'Back At Keyboard',
 'BBL': 'Be Back Later',
 'BBS': 'Be Back Soon',
 'BFN': 'Bye For Now',
 'B4N': 'Bye For Now',
 'BRB': 'Be Right Back',
 'BRT': 'Be Right There',
 'BTW': 'By The Way',
 'B4': 'Before',
 'CU': 'See You',
 'CUL8R': 'See You Later',
 'CYA': 'See You',
 'FAQ': 'Frequently Asked Questions',
 'FC': 'Fingers Crossed',
 'FWIW': 'For What It is Worth',
 'FYI': 'For Your Information',
 'GAL': 'Get A Life',
 'GG': 'Good Game',
 'GN': 'Good Night',
 'GMTA': 'Great Minds Think Alike',
 'GR8': 'Great',
 'G9': 'Genius',
 'IC': 'I See',
 'ICQ': 'I Seek you (also a chat program)',
 'ILU': 'I Love You',
 'IMHO': 'In My Honest/Humble Opinion',
 'IMO': 'In My Opinion',
 'IOW': 'In Other Words',
 'IRL': 'In Real Life',
 'KISS': 'Keep It Simple, Stupid',
 'LDR': 'Long Distance Relationship',
 'LMAO': 

In [23]:
# Removing the chatwords in conversations,whch does not add any signaficant values.
def chat_conversations(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)  

In [24]:
df['review'][5]

'probably my all-time favorite movie, a story of selflessness, sacrifice and dedication to a noble cause, but it\'s not preachy or boring. it just never gets old, despite my having seen it some 15 or more times in the last 25 years. paul lukas\' performance brings tears to my eyes, and bette davis, in one of her very few truly sympathetic roles, is a delight. the kids are, as grandma says, more like "dressed-up midgets" than children, but that only makes them more fun to watch. and the mother\'s slow awakening to what\'s happening in the world and under her own roof is believable and startling. if i had a dozen thumbs, they\'d all be "up" for this movie.'

In [25]:
df['review'].apply(chat_conversations)

0        one of the other reviewers has mentioned that ...
1        a wonderful little production. the filming tec...
2        i thought this was a wonderful way to spend ti...
3        basically there's a family where a little boy ...
4        petter mattei's "love in the time of money" is...
                               ...                        
19995    ok. for starters, taxi driver is amazing. this...
19996    it's sort of hard for me to say it, because i ...
19997    i still liked it though. warren beatty is only...
19998    we could still use black adder even today. ima...
19999    this so-called "documentary" tries to tell tha...
Name: review, Length: 19926, dtype: object

In [26]:
df.sample(5)

,review,sentiment
18232,"the greatest sin in life is being dull, and th...",negative
18487,okay like most steven seagal fans i know i not...,negative
3298,this movie is about a group of people who are ...,negative
11603,seeing les amants reguliers calls immediately ...,positive
3962,dead man's bounty (the film's american title) ...,negative


In [27]:
stopword = stopwords.words('english')

In [28]:
new_text =[]
def remove_stopwords(text):
    
    for word in text.split():
        if word in stopword:
            new_text.append('')
        else:
            new_text.append(word)
        x= new_text[:]
        new_text.clear()
        return " ".join(x)

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in stopword]).apply(lambda x:" ".join(x))

In [29]:
df['review'].apply(remove_stopwords)

0              one
1        wonderful
2          thought
3        basically
4           petter
           ...    
19995          ok.
19996         sort
19997        still
19998        could
19999    so-called
Name: review, Length: 19926, dtype: object

In [30]:
df['review'][3]

"basically there's family little boy (jake) thinks there's zombie closet & parents fighting time.this movie slower soap opera... suddenly, jake decides become rambo kill zombie.ok, first going make film must decide thriller drama! drama movie watchable. parents divorcing & arguing like real life. jake closet totally ruins film! expected see boogeyman similar movie, instead watched drama meaningless thriller spots.3 10 well playing parents & descent dialogs. shots jake: ignore them."

In [31]:
story = []
for doc in df['review']:
    new_sent = sent_tokenize(doc)
    for sent in new_sent:
        story.append(simple_preprocess(sent))

In [32]:
story[0:4]

[['one', 'reviewers', 'mentioned', 'watching', 'oz', 'episode', 'hooked'],
 ['right',
  'exactly',
  'happened',
  'me',
  'the',
  'first',
  'thing',
  'struck',
  'oz',
  'brutality',
  'unflinching',
  'scenes',
  'violence',
  'set',
  'right',
  'word',
  'go'],
 ['trust', 'me', 'show', 'faint', 'hearted', 'timid'],
 ['show', 'pulls', 'punches', 'regards', 'drugs', 'sex', 'violence']]

In [33]:
model = gensim.models.Word2Vec(window = 10, min_count = 2)

In [34]:
model.build_vocab(story)

In [35]:
model.train(story, total_examples = model.corpus_count, epochs = model.epochs)

(11809611, 12410520)

In [36]:
def document_vector(doc):
    doc  = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis = 0 )

In [37]:
document_vector(df['review'].values[0])

array([-0.07524946,  0.02527062,  0.02771504,  0.09052797,  0.3625489 ,
       -0.32307202, -0.10454083,  0.66457224, -0.00573746, -0.2691916 ,
        0.05220519, -0.23654957,  0.08528611,  0.02924959,  0.099882  ,
       -0.07473221,  0.1044746 , -0.49872193, -0.12614273, -0.41114384,
        0.23145434,  0.18367983,  0.15226912, -0.20496427, -0.01209292,
       -0.01777387, -0.1707525 , -0.06469508, -0.24417171,  0.16939503,
        0.45145687, -0.34558254, -0.10376392, -0.45089674, -0.14597653,
        0.34977502,  0.04551838, -0.36430725, -0.02076515, -0.6979752 ,
        0.04222639, -0.21655335, -0.1529218 , -0.09763652,  0.58474934,
        0.01591096, -0.35615012, -0.00565543, -0.03808333,  0.2368015 ,
        0.34480318, -0.06774708,  0.08583731, -0.07777703, -0.31604174,
        0.3286292 ,  0.12015914,  0.30595076, -0.47009048,  0.1060214 ,
        0.08942985,  0.21517919,  0.02336846, -0.00884246, -0.19642675,
        0.23778987,  0.14070313,  0.2648493 , -0.3419269 ,  0.42

In [38]:
x=[]
for doc in tqdm(df['review'].values):
    x.append(document_vector(doc))

100%|██████████████████████████████████████████████████████████████████████████| 19926/19926 [1:06:50<00:00,  4.97it/s]


In [39]:
x = np.array(x)

In [40]:

encoder = LabelEncoder()
y= encoder.fit_transform(df['sentiment'])

In [41]:
from  sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y, test_size=0.2,random_state = 3)

In [42]:
gnb = GaussianNB()
gnb.fit(x_train,y_train)
pred = gnb.predict(x_test)
print(pred)
print('the accuracy score for GNB is:', accuracy_score(pred,y_test))

[0 1 0 ... 1 0 0]
the accuracy score for GNB is: 0.7759658805820371


In [43]:
print('The precision score for GNB is:', precision_score(pred, y_test))
print('The Recall score for GNB is:', recall_score(y_test,pred))
print( 'The f1 score GNB is:', f1_score(pred, y_test))
print(confusion_matrix(pred,y_test))

The precision score for GNB is: 0.794425087108014
The Recall score for GNB is: 0.794425087108014
The f1 score GNB is: 0.7813953488372094
[[1497  413]
 [ 480 1596]]


In [44]:

rfc = RandomForestClassifier()

In [45]:
rfc.fit(x_train,y_train)
y_pred  =rfc.predict(x_test)
print(y_pred)
print('accuracy score for Random forest estimator is:', accuracy_score(y_pred, y_test))

[0 1 1 ... 1 0 0]
accuracy score for Random forest estimator is: 0.8050677370797792


In [46]:
print('The precision score for random forest is:', precision_score(y_pred, y_test))
print('The Recall score for random forest is:', recall_score(y_test,y_pred))
print( 'The f1 score random forest is:', f1_score(y_pred, y_test))
print(confusion_matrix(y_pred,y_test))

The precision score for random forest is: 0.8148332503733201
The Recall score for random forest is: 0.8148332503733201
The f1 score random forest is: 0.8081955072821525
[[1572  372]
 [ 405 1637]]
